In [27]:
import cx_Oracle
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup
import re
import urllib.request as ur
from matplotlib import font_manager,rc
import matplotlib.pyplot as plt
import seaborn as sns
import json

font_location = "C:\Windows\Fonts\malgun.ttf"
font_name = font_manager.FontProperties(fname=font_location).get_name()
rc('font',family=font_name)
plt.rcParams['axes.unicode_minus'] = False

In [83]:
conn = cx_Oracle.connect('open_source/1111@localhost:1521/xe')
cur = conn.cursor()

In [206]:
access_key = 860553

In [207]:
def get_request_url():
    url = 'http://api.jejuits.go.kr/api/infoParkingInfoList?code=860553'

    response = requests.get(url)
    return response.text


In [ ]:
def get_parsed_json(raw_json):
    all_data = []
    for record in raw_json['info']:
        all_data.append(
        {"id": record.get("id"),
         "name": record.get("name"),
         "addr": record.get("addr"),
         "x_crdn": record.get("x_crdn"),
         "y_crdn": record.get("y_crdn"),
         "park_day": record.get("park_day"),
         "wkdy_strt": record.get("wkdy_strt"),
         "wkdy_end": record.get("wkdy_end"),
         "lhdy_strt": record.get("lhdy_strt"),
         "lhdy_end": record.get("lhdy_end"),
         "basic_time": record.get("basic_time"),
         "basic_fare": record.get("basic_fare"),
         "add_time": record.get("add_time"),
         "add_fare": record.get("add_fare"),
         "whol_npls": record.get("whol_npls")
         }
        )
    return all_data,column_list

In [214]:
def json_to_df_info(parsed_json) :
    raw_str_json = get_request_url()
    if raw_str_json:
        # json.loads() 문자열 json을 실제 json(dict) 타입으로 변환
        raw_json = json.loads(raw_str_json)

    all_data = []
    column_list = ["id", "name", "addr","x_crdn","y_crdn","park_day","wkdy_strt","wkdy_end","lhdy_strt","lhdy_end","basic_time","basic_fare","add_time","add_fare","whol_npls"]

    for record in raw_json['info'] :
        row_data = []
        for column_data in column_list :
            row_data.append(record.get(column_data))
        all_data.append(row_data)
    return column_list, all_data

In [215]:
def Bus_info_collector():

    raw_str_json = get_request_url()

    if raw_str_json:
        raw_json = json.loads(raw_str_json)
    print(raw_json)
    column_list, all_data = json_to_df_info(raw_json)

    df = pd.DataFrame(all_data, columns=column_list)

    df_preprocessed = preprocessed_df(df)

    preprocessed_df_to_oracle(df_preprocessed)

In [216]:
Bus_info_collector()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [98]:
html = urllib.request.urlopen('http://api.jejuits.go.kr/api/infoParkingInfoList?code=860553')
soup=BeautifulSoup(html, 'html.parser')
soup

{"result":"success","info_cnt":10,"info":[{"id":"11111111","name":"서귀포매일올레시장","addr":"서귀포시 중앙로 62번길 18","x_crdn":126.56326295,"y_crdn":33.25031562,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":216},{"id":"16488201","name":"법원북측공영주차장","addr":"제주시 이도이동 1066","x_crdn":126.53534209,"y_crdn":33.49472463,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":91},{"id":"20019319","name":"산짓물공영주차장","addr":"제주시 건입동 1330","x_crdn":126.52872778,"y_crdn":33.51590058,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":71},{"id":"17680713","name":"이도2동공영주차장","addr":"제주시 오복3길 9 (이도이동 1052-2)","x_crdn":126.53494394,"y_crdn":33.

In [178]:
type(soup)

bs4.BeautifulSoup

In [170]:
corse=re.compile('([{]["]id["][:]["].*["][,]["]name["][:]["].*["][,]["]addr["][:]["].*["][,]["]x_crdn["]\d*.*add_fare["][:]\d*[,]["]whol_npls["][:]\d*[}])')
c=corse.findall(str(soup))
print(c)
#[.]\d*[,]["]y_crdn["][:]

['{"id":"11111111","name":"서귀포매일올레시장","addr":"서귀포시 중앙로 62번길 18","x_crdn":126.56326295,"y_crdn":33.25031562,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":216},{"id":"16488201","name":"법원북측공영주차장","addr":"제주시 이도이동 1066","x_crdn":126.53534209,"y_crdn":33.49472463,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":91},{"id":"20019319","name":"산짓물공영주차장","addr":"제주시 건입동 1330","x_crdn":126.52872778,"y_crdn":33.51590058,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":71},{"id":"17680713","name":"이도2동공영주차장","addr":"제주시 오복3길 9 (이도이동 1052-2)","x_crdn":126.53494394,"y_crdn":33.49673819,"park_day":"월화수목금토일","wkdy_strt

In [204]:
json_to_df_info(soup)

AttributeError: 'NavigableString' object has no attribute 'get'

In [186]:
type(c)

list

In [160]:
alldata = {"id":"11111111","name":"서귀포매일올레시장","addr":"서귀포시 중앙로 62번길 18","x_crdn":126.56326295,"y_crdn":33.25031562,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":216},{"id":"16488201","name":"법원북측공영주차장","addr":"제주시 이도이동 1066","x_crdn":126.53534209,"y_crdn":33.49472463,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":91},{"id":"20019319","name":"산짓물공영주차장","addr":"제주시 건입동 1330","x_crdn":126.52872778,"y_crdn":33.51590058,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":71},{"id":"17680713","name":"이도2동공영주차장","addr":"제주시 오복3길 9 (이도이동 1052-2)","x_crdn":126.53494394,"y_crdn":33.49673819,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":150},{"id":"17759313","name":"북수구공영주차장","addr":"제주시 일도일동 1230-3","x_crdn":126.52794054,"y_crdn":33.5146917,"park_day":"월화수목금토일","wkdy_strt":"080000","wkdy_end":"220000","lhdy_strt":"080000","lhdy_end":"220000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":51},{"id":"11251813","name":"신제주공영주차장","addr":"제주시 연동 신대로12길","x_crdn":126.49106817,"y_crdn":33.48893829,"park_day":"월화수목금","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":500},{"id":"17039715","name":"동문공설시장공영주차장","addr":"제주시 동문로4길 9 (일도일동 1104-2)","x_crdn":126.52819953,"y_crdn":33.51208935,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":264},{"id":"17385794","name":"서귀중앙공영주차빌딩","addr":"서귀포시 중앙로 54번길 17 (서귀동 291-63)","x_crdn":126.56221093,"y_crdn":33.2498566,"park_day":"월화수목금토일","wkdy_strt":"090000","wkdy_end":"180000","lhdy_strt":"090000","lhdy_end":"180000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":306},{"id":"17435963","name":"칠성골공영주차장","addr":"제주시 임항로 8 (삼도이동 1192-8)","x_crdn":126.525116,"y_crdn":33.516579,"park_day":"월화수목금토일","wkdy_strt":"120000","wkdy_end":"210000","lhdy_strt":"120000","lhdy_end":"210000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":80},{"id":"16489518","name":"인제공영주차장","addr":"제주시 고마로 19길 5 (일도이동 409-11)","x_crdn":126.54195004,"y_crdn":33.50479298,"park_day":"월화수목금토일","wkdy_strt":"120000","wkdy_end":"210000","lhdy_strt":"120000","lhdy_end":"210000","basic_time":30,"basic_fare":1000,"add_time":15,"add_fare":500,"whol_npls":125}

In [175]:
type(alldata)

tuple

In [172]:
alldata

({'id': '11111111',
  'name': '서귀포매일올레시장',
  'addr': '서귀포시 중앙로 62번길 18',
  'x_crdn': 126.56326295,
  'y_crdn': 33.25031562,
  'park_day': '월화수목금토일',
  'wkdy_strt': '090000',
  'wkdy_end': '180000',
  'lhdy_strt': '090000',
  'lhdy_end': '180000',
  'basic_time': 30,
  'basic_fare': 1000,
  'add_time': 15,
  'add_fare': 500,
  'whol_npls': 216},
 {'id': '16488201',
  'name': '법원북측공영주차장',
  'addr': '제주시 이도이동 1066',
  'x_crdn': 126.53534209,
  'y_crdn': 33.49472463,
  'park_day': '월화수목금토일',
  'wkdy_strt': '090000',
  'wkdy_end': '180000',
  'lhdy_strt': '090000',
  'lhdy_end': '180000',
  'basic_time': 30,
  'basic_fare': 1000,
  'add_time': 15,
  'add_fare': 500,
  'whol_npls': 91},
 {'id': '20019319',
  'name': '산짓물공영주차장',
  'addr': '제주시 건입동 1330',
  'x_crdn': 126.52872778,
  'y_crdn': 33.51590058,
  'park_day': '월화수목금토일',
  'wkdy_strt': '090000',
  'wkdy_end': '180000',
  'lhdy_strt': '090000',
  'lhdy_end': '180000',
  'basic_time': 30,
  'basic_fare': 1000,
  'add_time': 15,
  'add_f

In [169]:
parking_area_df = pd.DataFrame(alldata)
parking_area_df

,id,name,addr,x_crdn,y_crdn,park_day,wkdy_strt,wkdy_end,lhdy_strt,lhdy_end,basic_time,basic_fare,add_time,add_fare,whol_npls
0,11111111,서귀포매일올레시장,서귀포시 중앙로 62번길 18,126.563263,33.250316,월화수목금토일,090000,180000,090000,180000,30,1000,15,500,216
1,16488201,법원북측공영주차장,제주시 이도이동 1066,126.535342,33.494725,월화수목금토일,090000,180000,090000,180000,30,1000,15,500,91
2,20019319,산짓물공영주차장,제주시 건입동 1330,126.528728,33.515901,월화수목금토일,090000,180000,090000,180000,30,1000,15,500,71
3,17680713,이도2동공영주차장,제주시 오복3길 9 (이도이동 1052-2),126.534944,33.496738,월화수목금토일,090000,180000,090000,180000,30,1000,15,500,150
4,17759313,북수구공영주차장,제주시 일도일동 1230-3,126.527941,33.514692,월화수목금토일,080000,220000,080000,220000,30,1000,15,500,51
5,11251813,신제주공영주차장,제주시 연동 신대로12길,126.491068,33.488938,월화수목금,090000,180000,090000,180000,30,1000,15,500,500
6,17039715,동문공설시장공영주차장,제주시 동문로4길 9 (일도일동 1104-2),126.528200,33.512089,월화수목금토일,090000,180000,090000,180000,30,1000,15,500,264
7,17385794,서귀중앙공영주차빌딩,서귀포시 중앙로 54번길 17 (서귀동 291-63),126.562211,33.249857,월화수목금토일,090000,180000,090000,180000,30,1000,15,500,306
8,17435963,칠성골공영주차장,제주시 임항로 8 (삼도이동 1192-8),126.525116,33.516579,월화수목금토일,120000,210000,120000,210000,30,1000,15,500,80
9,16489518,인제공영주차장,제주시 고마로 19길 5 (일도이동 409-11),126.541950,33.504793,월화수목금토일,120000,210000,120000,210000,30,1000,15,500,125
